In [1]:

# Code from the next sources were used
#[4] A transformer chatbot tutorial with Tensorflow 2.0. Retrieved from https://medium.com/tensorflow/a-transformer-chatbot-tutorial-with-tensorflow-2-0-88bf59e66fe2
#[6] https://www.kaggle.com/maxwell110/beginner-s-guide-to-audio-data-2
#[7] https://www.kaggle.com/daisukelab/cnn-2d-basic-solution-powered-by-fast-ai

# We load the requiere libraries
import numpy as np
import pandas as pd 
import os
# import git
import librosa
import matplotlib.pyplot as plt
import gc

from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import zipfile

tqdm.pandas()


/Users/meghana/dev/miniconda3/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Users/meghana/dev/miniconda3/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Users/meghana/dev/miniconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be 

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

In [3]:
# #We load csv with files name. We will use for reading the data
# df = pd.read_csv('/home/facudeza/train_curated.csv')
# #df_noisy = pd.read_csv('../input/freesound-audio-tagging-2019/train_noisy.csv')

# #train=pd.concat([df, df_noisy], axis=0)

# test_df = pd.read_csv('/home/facudeza//sample_submission.csv')

# len(test_df)


In [4]:
import pandas as pd       
import os 
import math 
import numpy as np
import matplotlib.pyplot as plt  
import IPython.display as ipd  # To play sound in the notebook
import librosa
import librosa.display
import time
import warnings
import pickle as pickle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
SAMPLE_RATE = 22050
warnings.filterwarnings('ignore')
df = pd.read_csv("speakers_all.csv")
df.drop(df.columns[9:12],axis = 1, inplace = True)
print(df.columns)
df.describe()

Index(['age', 'age_onset', 'birthplace', 'filename', 'native_language', 'sex',
       'speakerid', 'country', 'file_missing?'],
      dtype='object')


,age,age_onset,speakerid
count,2172.000000,2172.000000,2172.000000
mean,33.117173,8.833333,1088.449355
std,14.453039,8.451127,628.420329
min,0.000000,0.000000,1.000000
25%,22.000000,0.000000,543.750000
50%,28.000000,8.000000,1088.500000
75%,41.000000,13.000000,1632.250000
max,97.000000,86.000000,2176.000000


In [37]:
# start_time = time.time()

# def extract_features_from_pkl(fname):
#     try:
#         path = 'features/' + fname +".pkl"
#         extracted_data = pd.read_pickle(path)
#         data, sampling_rate = extracted_data["data"], extracted_data["sampling_rate"]
#         duration = extracted_data["duration"]
#         mfcc = extracted_data["mfcc"]
#         melspec = extracted_data["melspec"]
#         y_harmonic, y_percussive = extracted_data["y_harmonic"],extracted_data["y_percussive"]
#         melspec_h = extracted_data["melspec_h"]
#         log_h = extracted_data["log_h"]
#         melspec_p = extracted_data["melspec_p"]
#         log_p = extracted_data["log_p"]
#         zero_crossing_rate = extracted_data["zero_crossing_rate"]
#         spectral_rolloff = extracted_data["spectral_rolloff"]
#         spectral_centroid = extracted_data["spectral_centroid"]
#         spectral_contrast = extracted_data["spectral_contrast"]
#         spectral_bandwidth = extracted_data["spectral_bandwidth"]
#         chroma_cqt = extracted_data["chroma_cqt"]
#         return (duration,mfcc,melspec,melspec_h,melspec_p,y_harmonic, y_percussive,log_h
#                 ,log_p,data,sampling_rate,zero_crossing_rate,spectral_rolloff,spectral_centroid
#                 ,spectral_contrast,spectral_bandwidth,chroma_cqt)
#     except Exception:
#         return 

# durations,mfccs,melspecs,melspec_hs,melspec_ps,y_harmonics,y_percussives = [],[],[],[],[],[],[]
# log_hs,log_ps,datas,sampling_rates = [],[],[],[]
# zero_crossing_rates,spectral_rolloffs, spectral_centroids = [],[],[]
# spectral_contrasts,spectral_bandwidths,chroma_cqts = [],[],[]
# for _, row in df.iterrows():
#     print(_)
#     filename = row['filename']
#     val = extract_features_from_pkl(filename)
#     if val:
#         durations.append(val[0])
#         mfccs.append(val[1])
#         melspecs.append(val[2])
#         melspec_hs.append(val[3])
#         melspec_ps.append(val[4])
#         y_harmonics.append(val[5])
#         y_percussives.append(val[6])
#         log_hs.append(val[7])
#         log_ps.append(val[8])
#         datas.append(val[9])
#         sampling_rates.append(val[10])
#         zero_crossing_rates.append(val[11])
#         spectral_rolloffs.append(val[12])
#         spectral_centroids.append(val[13])
#         spectral_contrasts.append(val[14])
#         spectral_bandwidths.append(val[15])
#         chroma_cqts.append(val[16])
#     else:
#         durations.append(None)
#         mfccs.append(None)
#         melspecs.append(None)
#         melspec_hs.append(None)
#         melspec_ps.append(None)
#         y_harmonics.append(None)
#         y_percussives.append(None)
#         log_hs.append(None)
#         log_ps.append(None)
#         datas.append(None)
#         sampling_rates.append(None)
#         zero_crossing_rates.append(None)
#         spectral_rolloffs.append(None)
#         spectral_centroids.append(None)
#         spectral_contrasts.append(None)
#         spectral_bandwidths.append(None)
#         chroma_cqts.append(None)

# print("time taken "+str((time.time() - start_time)/60))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087


In [38]:
# df["duration"] = durations
# df["mfcc"] = mfccs
# df["melspec"] = melspecs
# df["melspec_h"] = melspec_hs
# df["melspec_p"] = melspec_ps
# df["y_harmonic"] = y_harmonics
# df["y_percussive"] = y_percussives
# df["melspec_harmonic"] = log_hs
# df["melspec_percussive"] = log_ps
# df["librosa_data"] = datas
# df["librosa_sampling_rate"] = sampling_rates
# df["zero_crossing_rate"] = zero_crossing_rates
# df["spectral_rolloff"] = spectral_rolloffs
# df["spectral_centroid"] = spectral_centroids
# df["spectral_contrast"] = spectral_contrasts
# df["spectral_bandwidth"] = spectral_bandwidths
# df["chroma_cqt"] = chroma_cqts
# df = df.dropna()
# df.head()
# df.drop(columns=['age_onset','filename','speakerid','file_missing?'],inplace=True)
# df.head()

,age,birthplace,native_language,sex,country,duration,mfcc,melspec,melspec_h,melspec_p,...,melspec_harmonic,melspec_percussive,librosa_data,librosa_sampling_rate,zero_crossing_rate,spectral_rolloff,spectral_centroid,spectral_contrast,spectral_bandwidth,chroma_cqt
32,27.0,"virginia, south africa",afrikaans,female,south africa,20.772290,"[[-395.00208, -387.00363, -377.9013, -392.9432...","[[0.09874383, 0.25136992, 0.25506964, 0.223414...","[[0.048649844, 0.11224768, 0.10517518, 0.06032...","[[0.016398141, 0.033163264, 0.038705193, 0.063...",...,"[[-5.995369, -5.995369, -5.995369, -5.995369, ...","[[-30.363499, -29.586853, -28.244614, -23.9563...","[0.0071707093, 0.010596839, 0.00808429, 0.0093...",22050.0,"[0.01611328125, 0.0244140625, 0.033203125, 0.0...","[2950.048828125, 2971.58203125, 3068.481445312...","[1530.4223975774694, 1423.8069150707834, 1320....","[36.50567083862842, 14.012752078187578, 10.362...","[2463.8000385430314, 2273.238307315652, 2083.8...","[[0.8212306011679301, 0.8040046693991846, 0.78..."
33,40.0,"pretoria, south africa",afrikaans,male,south africa,21.961497,"[[-560.3812, -560.3812, -560.3812, -560.3812, ...","[[2.1711987e-06, 2.455684e-06, 2.3796056e-06, ...","[[2.1903074e-06, 2.4361193e-06, 2.391069e-06, ...","[[4.3559062e-10, 2.1939425e-10, 1.1125337e-09,...",...,"[[-24.149624, -24.149624, -24.149624, -24.1496...","[[-24.416653, -24.416653, -24.416653, -24.4166...","[-1.8169361e-05, -2.4786286e-05, -8.806779e-06...",22050.0,"[0.025390625, 0.041015625, 0.05029296875, 0.04...","[8990.1123046875, 9108.544921875, 9248.5107421...","[5123.089311931954, 5264.551708269, 5429.44865...","[32.56676585595034, 28.16572518059867, 24.7029...","[3348.660970144079, 3348.077664928403, 3389.43...","[[0.9168646531128495, 0.6017439302030426, 1.0,..."
34,43.0,"pretoria, transvaal, south africa",afrikaans,male,south africa,26.880000,"[[-441.16895, -439.80875, -430.03815, -406.936...","[[0.006251498, 0.005871368, 0.0032841666, 0.00...","[[0.00229122, 0.0017953095, 0.0010293978, 0.00...","[[0.0010243187, 0.0012062755, 0.00065918657, 0...",...,"[[-17.287262, -17.287262, -17.287262, -17.2872...","[[-15.318008, -15.318008, -15.318008, -15.3180...","[0.0019851814, 0.0027917789, 0.0013547958, 0.0...",22050.0,"[0.0244140625, 0.0322265625, 0.05126953125, 0....","[3079.248046875, 2627.05078125, 2616.284179687...","[1465.0935000653856, 1361.8608658509977, 1347....","[22.474867417512268, 26.549838842792123, 29.65...","[2159.5564308533862, 2036.7410293218215, 1875....","[[0.9405656123995829, 0.8434395912814198, 0.93..."
35,26.0,"pretoria, south africa",afrikaans,male,south africa,23.471202,"[[-529.75555, -529.76184, -529.74896, -529.790...","[[0.0005168101, 0.0005084191, 0.00052578124, 0...","[[0.00051670696, 0.0005085286, 0.00052552565, ...","[[5.3004323e-11, 8.718523e-11, 3.5090522e-10, ...",...,"[[-18.92363, -18.92363, -18.92363, -18.92363, ...","[[-17.007862, -17.007862, -17.007862, -17.0078...","[0.00025531906, 0.0003802524, 0.00032837203, 0...",22050.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8182.6171875, 8322.5830078125, 8268.75, 7967....","[2836.1443828141787, 2981.4239632758927, 2943....","[31.146817865859134, 33.83682462665064, 35.432...","[3677.497426953188, 3705.7593851497895, 3685.5...","[[0.43540816224779805, 0.43058722215309736, 0...."
36,19.0,"cape town, south africa",afrikaans,male,south africa,20.252426,"[[-509.4199, -479.27243, -467.60425, -471.0367...","[[0.595527, 2.648835, 4.051913, 2.793845, 0.93...","[[0.40962306, 1.6456182, 3.0860555, 2.5644944,...","[[0.029812008, 0.13252573, 0.09091881, 0.00693...",...,"[[-7.7523117, 4.3265815, 9.7880745, 8.180036, ...","[[-30.512177, -17.553995, -20.826927, -35.1874...","[1.7647135e-05, -1.7076701e-06, -9.886354e-06,...",22050.0,"[0.0126953125, 0.01513671875, 0.0166015625, 0....","[1668.8232421875, 1604.2236328125, 1701.123046...","[808.4449193168485, 717.8543169790694, 817.630...","[20.726607494551942, 19.790129878017588, 21.93...","[1641.2152764813704, 1536.64683

In [5]:
start_time = time.time()

def extract_mfcc_melspecs_from_pkl(fname):
    try:
        path = 'features/' + fname +".pkl"
        extracted_data = pd.read_pickle(path)
        data, sampling_rate = extracted_data["data"], extracted_data["sampling_rate"]
        duration = extracted_data["duration"]
        mfcc = extracted_data["mfcc"]
        melspec = extracted_data["melspec"]
        return (duration,mfcc,melspec,data)
    except Exception:
        return 

durations,mfccs,melspecs,datas = [],[],[],[]
for _, row in df.iterrows():
    print(_)
    filename = row['filename']
    val = extract_mfcc_melspecs_from_pkl(filename)
    if val:
        durations.append(val[0])
        mfccs.append(val[1])
        melspecs.append(val[2])
        datas.append(val[3])
    else:
        durations.append(None)
        mfccs.append(None)
        melspecs.append(None)
        datas.append(None)

df["duration"] = durations
df["mfcc"] = mfccs
df["melspec"] = melspecs
df["librosa_data"] = datas


print("time taken "+str((time.time() - start_time)/60))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068


In [6]:

labels = np.sort(np.unique(df['native_language'].values))
num_class = len(labels)
c2i = {}
i2c = {}
for i, c in enumerate(labels):
    c2i[c] = i
    i2c[i] = c


def test(index):
    global num_class
    labels_array = np.zeros((num_class))
    labels_array[index] = 1
    return labels_array

one_hot_labels = [test(c2i[x]) for x in df['native_language'].values]
df["native_lang_c2i"] = one_hot_labels
df["native_lang_c2i"].head()
for f in labels:
    df[f] = 0.0
    #train_noisy[f] = 0.0

df[labels] = one_hot_labels
df.tail()

# def split_and_label(rows_labels):
    
#     row_labels_list = []
#     for row in rows_labels:
#         row_labels = row.split(',')
#         labels_array = np.zeros((80))
        
#         for label in row_labels:
#             index = label_mapping[label]
#             labels_array[index] = 1
        
#         row_labels_list.append(labels_array)
    
#     return row_labels_list


# train_curated_labels = split_and_label(df['labels'])

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?,duration,...,wolof,wu,xasonga,xiang,yakut,yapese,yiddish,yoruba,yupik,zulu
2167,46.0,5.0,"lagos, nigeria",yoruba3,yoruba,female,766,nigeria,False,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2168,46.0,12.0,"lagos, nigeria",yoruba4,yoruba,male,851,nigeria,False,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2169,47.0,2.0,"ibadan, nigeria",yoruba5,yoruba,female,2023,nigeria,False,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2170,31.0,1.0,"bethel, alaska, usa",yupik1,yupik,female,571,usa,False,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2171,24.0,14.0,"bulawayo, zimbabwe",zulu1,zulu,female,406,zimbabwe,False,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
df = df.dropna()

In [9]:
# We get the labels from the test data, we start from 1 because 0 is the file name
# Also, test set only has 80 classes and want to predict according to that.
# Of course, you could create 
# From here we will create Y target

# label_columns = test_df.columns[1:]
# label_mapping = dict((label, index) for index, label in enumerate(label_columns))


In [10]:
## We can see that several targets are in same space. We need to reshape it into a hot encode format
# print(df['labels'][:10])

In [11]:
# def split_and_label(rows_labels):
    
#     row_labels_list = []
#     for row in rows_labels:
#         row_labels = row.split(',')
#         labels_array = np.zeros((80))
        
#         for label in row_labels:
#             index = label_mapping[label]
#             labels_array[index] = 1
        
#         row_labels_list.append(labels_array)
    
#     return row_labels_list


# train_curated_labels = split_and_label(df['labels'])


In [12]:
# for f in label_columns:
#     df[f] = 0.0
#     #train_noisy[f] = 0.0

# df[label_columns] = train_curated_labels
# #train_noisy[label_columns]   = train_noisy_labels

# df['num_labels'] = df[label_columns].sum(axis=1)
# #train_noisy['num_labels']   = train_noisy[label_columns].sum(axis=1)

# #train_curated['path'] = '../input/train_curated/'+train_curated['fname']
# #train_noisy  ['path'] = '../input/train_noisy/'+train_noisy['fname']

# df.head()



In [13]:
# unzipping our files


# with zipfile.ZipFile('/home/facudeza/train_curated.zip', 'r') as zip_ref:
#     zip_ref.extractall('/home/jupyter/Untitled Folder')

In [14]:
#import zipfile
#with zipfile.ZipFile('../input/freesound-audio-tagging-2019/train_noisy.zip', 'r') as zip_ref:
 #   zip_ref.extractall('../output/kaggle/working/train')

    #


In [15]:
## We define this parameters for our audio processing

class conf:
    # Preprocessing settings
    sampling_rate = 44100
    duration = 4
    hop_length = 300
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    samples = sampling_rate * duration




In [16]:
# train_path = '/home/jupyter/Untitled Folder'
#train_noisy_path = '../input/train_noisy/'


In [17]:
## We read the audio data, and trim or padded, if it is needed
## This code was borrowed from https://www.kaggle.com/christofhenkel/keras-baseline-lstm-attention-5-fold

# def read_audio(conf, pathname, trim_long_data):
#     y, sr = librosa.load(pathname, sr=conf.sampling_rate)
#     # trim silence
#     if 0 < len(y): # workaround: 0 length causes error
#         y, _ = librosa.effects.trim(y) # trim, top_db=default(60)
#     # make it unified length to conf.samples
#     if len(y) > conf.samples: # long enough
#         if trim_long_data:
#             y = y[0:0+conf.samples]
#     else: # pad blank
#         padding = conf.samples - len(y)    # add padding at both ends
#         offset = padding // 2
#         y = np.pad(y, (offset, conf.samples - len(y) - offset), 'constant')
#     return y


# ## We convert our raw audio into a mel spectogram
# def audio_to_melspectrogram(conf, audio):
#     spectrogram = librosa.feature.melspectrogram(audio, 
#                                                  sr=conf.sampling_rate,
#                                                  n_mels=conf.n_mels,
#                                                  hop_length=conf.hop_length,
#                                                  n_fft=conf.n_fft,
#                                                  fmin=conf.fmin,
#                                                  fmax=conf.fmax)
#     spectrogram = librosa.power_to_db(spectrogram)
#     spectrogram = spectrogram.astype(np.float32)
#     return spectrogram

# #This implement the previous two function for one file
# def read_as_melspectrogram(conf, pathname, trim_long_data, debug_display=False):
#     x = read_audio(conf, pathname, trim_long_data)
#     mels = audio_to_melspectrogram(conf, x)
#     if debug_display:
#         IPython.display.display(IPython.display.Audio(x, rate=conf.sampling_rate))
#         show_melspectrogram(conf, mels)
#     return mels

# #This generalizes for all the audio files you have
# def convert_wav_to_image(df, source):
#     X = []
#     for i, row in tqdm_notebook(df.iterrows()):
#         try:
#             x = read_as_melspectrogram(conf, source[0]+'/'+str(row.fname), trim_long_data=True)
#         except:
#             x = read_as_melspectrogram(conf, source[1]+'/'+str(row.fname), trim_long_data=True)

#         X.append(x.transpose())
#     return X

In [20]:
xmin,ymin = 128,float("inf")
for _,row in df.iterrows():
    x,y = row["melspec"].shape
    xmin,ymin = min(x,xmin),min(y,ymin)
print(xmin,ymin)
    

128 646


In [21]:
df["librosa_data"].iloc[32].shape

(964514,)

In [22]:
df["melspec"].iloc[32][:,:680].shape

(128, 646)

In [23]:
# def trim_data(y,trim_long_data):
#     print(len(y),conf.samples)
#     if len(y) > conf.samples: # long enough
#         print("if")
#         if trim_long_data:
#             y = y[0:0+conf.samples]
#     else: # pad blank
#         print("else")
#         padding = conf.samples - len(y)    # add padding at both ends
#         offset = padding // 2
#         y = np.pad(y, (offset, conf.samples - len(y) - offset), 'constant')
#     print(y)
#     return y



def get_melspec(df):
    X = []
    for i, row in df.iterrows():
        x = row["melspec"]
#         x = x[:,:680]
        spectrogram = librosa.power_to_db(x)
        spectrogram = spectrogram.astype(np.float32)
        X.append(x.transpose())
    return X

X = np.array(get_melspec(df))

In [24]:
X.shape

(2133, 646, 128)

In [25]:
#We get all our spectrograms
# X = np.array(convert_wav_to_image(df, source=[train_path]))


In [26]:
#np.savez('/home/jupyter/curated_subset.npz',X=X)

In [27]:
# X=np.array(np.load('/home/jupyter/curated_subset.npz')['X'])
# X.shape

In [57]:
df.iloc[:,14:]

,afrikaans,agni,akan,albanian,amazigh,amharic,arabic,armenian,ashanti,azerbaijani,...,wolof,wu,xasonga,xiang,yakut,yapese,yiddish,yoruba,yupik,zulu
32,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [36]:
y_df = df.iloc[:,14:].copy()

In [37]:
Y = np.array(y_df)

In [38]:
#df['number_labels'].mean()
# Y=np.array(df.drop(['fname', 'labels','num_labels'], axis=1))
# Y.shape

In [39]:
def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask zero out padding tokens.
  if mask is not None:
    logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(logits, axis=-1)

  return tf.matmul(attention_weights, value)

In [40]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    outputs = self.dense(concat_attention)

    return outputs

In [41]:
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [42]:
# This allows to the transformer to know where there is real data and where it is padded
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [43]:
def encoder_layer(units, d_model, num_heads, dropout,name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None,d_model ), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [44]:
def encoder(time_steps,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            projection,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
  
  if projection=='linear':
    ## We implement a linear projection based on Very Deep Self-Attention Networks for End-to-End Speech Recognition. Retrieved from https://arxiv.org/abs/1904.13377
    projection=tf.keras.layers.Dense( d_model,use_bias=True, activation='linear')(inputs)
    print('linear')
  
  else:
    projection=tf.identity(inputs)
    print('none')
   
  projection *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  projection = PositionalEncoding(time_steps, d_model)(projection)

  outputs = tf.keras.layers.Dropout(rate=dropout)(projection)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])
 
 
  

 
  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [45]:
def transformer(time_steps,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                output_size,
                projection,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  
  
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(tf.dtypes.cast(
          
    #Like our input has a dimension of length X d_model but the masking is applied to a vector
    # We get the sum for each row and result is a vector. So, if result is 0 it is because in that position was masked      
    tf.math.reduce_sum(
    inputs,
    axis=2,
    keepdims=False,
    name=None
), tf.int32))
  

  enc_outputs = encoder(
      time_steps=time_steps,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      projection=projection,
      name='encoder'
  )(inputs=[inputs, enc_padding_mask])

  #We reshape for feeding our FC in the next step
  outputs=tf.reshape(enc_outputs,(-1,time_steps*d_model))
  
  #We predict our class
  outputs = tf.keras.layers.Dense(units=output_size,use_bias=True,activation='softmax', name="outputs")(outputs)

  return tf.keras.Model(inputs=[inputs], outputs=outputs, name='audio_class')

In [46]:
#We get our train and test set
X_train,X_test, Y_train, Y_test =train_test_split(X,Y, test_size=0.2, random_state=27)
X_train,X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=0.2,random_state=27)



In [55]:
Y_train.shape

(1364, 214)

In [47]:
print(X_train.shape,X_val.shape,X_test.shape)

(1364, 646, 128) (342, 646, 128) (427, 646, 128)


In [48]:
projection=['linear','none']
accuracy=[]
proj_implemented=[]

In [58]:
for i in projection:
  NUM_LAYERS = 2
  D_MODEL = X.shape[2]
  NUM_HEADS = 4
  UNITS = 1024
  DROPOUT = 0.1
  TIME_STEPS= X.shape[1]
  OUTPUT_SIZE=214
  EPOCHS = 10
  EXPERIMENTS=10

  
  for j in range(EXPERIMENTS):
    
    
    model = transformer(time_steps=TIME_STEPS,
      num_layers=NUM_LAYERS,
      units=UNITS,
      d_model=D_MODEL,
      num_heads=NUM_HEADS,
      dropout=DROPOUT,
      output_size=OUTPUT_SIZE,  
      projection=i  )
    
    model.compile(optimizer=tf.keras.optimizers.Adam(0.000001), loss='categorical_crossentropy', metrics=['accuracy'])


    history=model.fit(X_train,Y_train, epochs=EPOCHS, validation_data=(X_test, Y_test))

    accuracy.append(max(history.history['val_accuracy']))
      
    proj_implemented.append(i)
    
    del model
    
    del history
      
   


linear
Train on 1364 samples, validate on 427 samples
Epoch 1/10
1364/1364 [==============================] - 391s 287ms/sample - loss: 5.9495 - accuracy: 0.0073 - val_loss: 5.3855 - val_accuracy: 0.0211
Epoch 2/10
1364/1364 [==============================] - 376s 276ms/sample - loss: 5.2094 - accuracy: 0.0359 - val_loss: 4.7870 - val_accuracy: 0.0843
Epoch 3/10
1364/1364 [==============================] - 362s 265ms/sample - loss: 4.6736 - accuracy: 0.0997 - val_loss: 4.4771 - val_accuracy: 0.1663
Epoch 4/10
1364/1364 [==============================] - 462s 339ms/sample - loss: 4.3706 - accuracy: 0.1437 - val_loss: 4.3619 - val_accuracy: 0.2248
Epoch 5/10
1364/1364 [==============================] - 583s 427ms/sample - loss: 4.2469 - accuracy: 0.1635 - val_loss: 4.3213 - val_accuracy: 0.2295
Epoch 6/10
1364/1364 [==============================] - 631s 463ms/sample - loss: 4.1770 - accuracy: 0.1686 - val_loss: 4.2985 - val_accuracy: 0.2389
Epoch 7/10
1364/1364 [========================

1364/1364 [==============================] - 312s 228ms/sample - loss: 4.4701 - accuracy: 0.1789 - val_loss: 4.2587 - val_accuracy: 0.2482
Epoch 4/10
1364/1364 [==============================] - 313s 230ms/sample - loss: 4.2535 - accuracy: 0.1906 - val_loss: 4.1988 - val_accuracy: 0.2389
Epoch 5/10
1364/1364 [==============================] - 313s 229ms/sample - loss: 4.2157 - accuracy: 0.1752 - val_loss: 4.1774 - val_accuracy: 0.2319
Epoch 6/10
1364/1364 [==============================] - 311s 228ms/sample - loss: 4.0837 - accuracy: 0.1701 - val_loss: 4.1686 - val_accuracy: 0.2365
Epoch 7/10
1364/1364 [==============================] - 312s 229ms/sample - loss: 3.9977 - accuracy: 0.1870 - val_loss: 4.1629 - val_accuracy: 0.2436
Epoch 8/10
1364/1364 [==============================] - 348s 255ms/sample - loss: 3.9935 - accuracy: 0.1877 - val_loss: 4.1567 - val_accuracy: 0.2412
Epoch 9/10
1364/1364 [==============================] - 342s 250ms/sample - loss: 3.8986 - accuracy: 0.1884 - v

KeyboardInterrupt: 

In [0]:
accuracy=pd.DataFrame(accuracy, columns=['accuracy'])
proj_implemented=pd.DataFrame(proj_implemented, columns=['projection'])
results=pd.concat([accuracy,proj_implemented],axis=1)


In [0]:
results.groupby('projection').mean()


,accuracy
projection,
linear,0.102213
none,0.086821


In [0]:
results.groupby('projection').std()

,accuracy
projection,
linear,0.017835
none,0.018532


In [89]:
df["duration"].min()

15.768299319727891